In [4]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
import torch

In [6]:
data = pd.read_csv('sample_data/data.csv')
data.head()

,Age,Gender,Ethnicity,Region,Socioeconomic Status,Symptom Description,Symptom Severity,Duration of Symptoms (days),Additional Symptoms,Chronic Condition,Allergies,Previous Visits,Potential Condition,Suggested Action,Insurance Status
0,25,female,caucasian,urban,low,persistent cough and shortness of breath,1.0,10.0,fatigue,asthma,penicillin,4.0,food poisoning,go to emergency,yes
1,53,male,asian,rural,low,fatigue and weakness,7.0,21.0,fatigue,diabetes,penicillin,6.0,muscle strain,go to emergency,yes
2,53,female,hispanic,suburban,low,frequent urination,9.0,23.0,fatigue,asthma,penicillin,8.0,diabetes,take home care measures,yes
3,79,female,other,rural,middle,chest pain,7.0,15.0,none,asthma,peanuts,7.0,muscle strain,take home care measures,no
4,79,male,other,urban,high,mild headache,8.0,22.0,none,diabetes,pollen,1.0,covid-19,visit doctor,no


In [7]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['Age', 'Gender', 'Ethnicity', 'Region', 'Socioeconomic Status', 'Symptom Description', 'Symptom Severity', 'Duration of Symptoms (days)', 'Additional Symptoms', 'Chronic Condition', 'Allergies', 'Previous Visits', 'Potential Condition', 'Suggested Action', 'Insurance Status'],
    num_rows: 182
})

In [9]:
def tokenization_function(examples):
    return tokenizer(examples['Symptom Description'], padding='max_length', truncation=True)

tokenization_dataset =dataset.map(tokenization_function, batched=True)

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

In [10]:
train_test_split = tokenization_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',           # model checkpoints will be saved here
    run_name='DocAssist_Assistance',    # specify a unique run name to avoid the warning
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    warmup_steps=500,                 # number of warmup steps
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory to save logs
    logging_steps=10,                 # log every 10 steps
    report_to=None
)


In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
from transformers import Trainer

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./model/biobert_finetuned')
tokenizer.save_pretrained('./model/biobert_finetuned')

In [ ]:
# Example symptom description
sample_input = "I have a persistent cough and mild fever."

# Tokenize sample input
inputs = tokenizer(sample_input, return_tensors="pt", padding=True, truncation=True)

# Run the model and get predictions
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print(f"Predicted label for the input: {predictions.item()}")
